In [1]:
from autostep3 import better_aubio, util
from threading import Thread
import soundfile as sf
import sounddevice as sd
import numpy as np

ModuleNotFoundError: No module named 'util'

Separate the drums from the song

In [10]:
drums_path, _ = util.separate_drums("test_audio/hyperpop.mp3")

Load the separated drums for playback

In [11]:
drums, samplerate = sf.read(drums_path, dtype='float32')
num_samples = sf.info(drums_path).frames

Generate onsets based on the drums
- Change the onset method to experiment

In [7]:
onsets = better_aubio.onset(drums_path, method='complex')

Generate the "audible onsets" track; on every onset you will hear a very short clipping sound

In [9]:
# god numpy is FAST
audible_onsets = np.float32(np.isin(np.arange(num_samples), onsets))

Create 2 audio streams using `sounddevice` to play both tracks simultaneously

In [10]:
song_stream = sd.OutputStream(channels=2, samplerate=samplerate)
onsets_stream = sd.OutputStream(channels=1, samplerate=samplerate)

The streams need to be multithreaded, since `OutputStream.write()` is a blocking call.
This cell begins audio playback.

In [11]:
song_stream.start()
onsets_stream.start()
Thread(target=lambda: song_stream.write(drums)).start()
Thread(target=lambda: onsets_stream.write(audible_onsets)).start()

Run this cell to stop the streams, in case errors occur in the main thread

In [12]:
song_stream.stop()
onsets_stream.stop()